In [ ]:
import numpy as np
import pandas as pd
import re
from math import sin, cos

In [ ]:
def pull_time(string, unit, round_range):
    pattern = [r'\d\d:\d\d:\d\d\.\d+',r'\d\d:\d\d:\d\d']
    for i in pattern:
        time_str = re.search(i, string)
        if time_str is not None:
            time_str = time_str.group()
            break
    time = list(map(int,re.findall(r'\d+',time_str)))
    if len(time) == 4:
        ms = time[-1] + time[-2]*1000 + time[-3]*60*1000 + time[-4]*60*60*1000
        if unit == 'h':
            ans = 1.*ms/(1000*60*60)
        elif unit == 'm':
            ans = 1.*ms/(1000*60)
        elif unit == 's':
            ans = 1.*ms/(1000)
        elif unit == 'ms':
            ans = 1.*ms
    elif len(time) == 3:
        s = time[-1] + time[-2]*60 + time[-3]*60*60
        if unit == 'h':
            ans = 1.*s/(60*60)
        elif unit == 'm':
            ans = 1.*s/(60)
        elif unit == 's':
            ans = 1.*s
        elif unit == 'ms':
            ans = 1.*s*1000
    return round(ans,round_range)

In [ ]:
def decompose_vel(velocity,angle): #Разложение скорости по компонентам с учетом углового смещения
    offset_angle = -90 #Градусов
    return [velocity*np.cos(np.deg2rad(angle+offset_angle)),velocity*np.sin(np.deg2rad(angle+offset_angle))]

In [ ]:
class Gaussian:
    def __init__(self, mu=None, Sigma=None): #Если одномерный, то sigma2
        self.mu = mu
        self.Sigma = Sigma

    def get(self):
        return [self.mu, self.sigma2]

    def solve_eq(self, x):
        return 1/np.sqrt(2 * np.pi * self.sigma2)    *    np.exp(-0.5 * (x - self.mu) ** 2 / self.sigma2)

    def fill(self,n=100):
        x = np.linspace(self.mu-3*np.sqrt(self.sigma2),self.mu+3*np.sqrt(self.sigma2),n)
        y = []
        for i in x:
            y.append(self.solve_eq(i))
        y = np.array(y)
        return(np.array([x,y]))

    def mul(self, lst_gausses):
        if len(lst_gausses) == 0:
            return
        f_mu = lst_gausses[0].mu
        f_sigma2 = lst_gausses[0].sigma2
        n = len(lst_gausses)
        for i in range(1, n):
            g_mu = lst_gausses[i].mu
            g_sigma2 = lst_gausses[i].sigma2

            K = f_sigma2 / (f_sigma2 + g_sigma2)

            f_mu = f_mu + K * (g_mu - f_mu)
            f_sigma2 = f_sigma2 - K * f_sigma2

        self.mu = f_mu
        self.sigma2 = f_sigma2

In [ ]:
old_df = pd.read_csv('data1.csv')
col = old_df.columns.tolist()
old_df = old_df.rename(columns={i: i.strip() for i in col})

df = pd.DataFrame(old_df.iloc [[0]])


time = str(df[["Device Time"]])
df["Time(s)"] = pull_time(time, "s", 2)
if df.loc[0, "Engine RPM(rpm)"] == "-":
  df.loc[0, "Engine RPM(rpm)"] = np.nan



for i in range(1, old_df.shape[0]):
  df.loc[i] = old_df.loc[i]
  if old_df.at[i-1,'GPS Time'] == old_df.at[i,'GPS Time']:
    df.loc[i,"GPS Time"] = np.nan
    df.loc[i,"Longitude"] = np.nan
    df.loc[i,"Latitude"] = np.nan
    df.loc[i,"GPS Speed (Meters/second)"] = np.nan
    df.loc[i,"Speed (GPS)(km/h)"] = np.nan
  time = str(old_df.loc[i, "Device Time"])
  df.loc[i,"Time(s)"] = pull_time(time, "s", 2)
  n = old_df.loc[i, "Engine RPM(rpm)"]
  if n == "-":
    df.loc[i, "Engine RPM(rpm)"] = np.nan
df["Engine RPM(rpm)"] = df["Engine RPM(rpm)"].astype(float)



old_bearing = 0
first_zeros = True

for i in range(1, df.shape[0]):
  gps = df.loc[i,"GPS Time"]
  bearing = df.loc[i,"Bearing"]
  old_bearing = df.loc[i-1,"Bearing"]
  if abs(bearing - old_bearing) >= 120:
    if not first_zeros:
      df.loc[i,"Bearing"] = old_bearing
    else:
      first_zeros = False

for i in range(df.shape[0]):
  lst = dict(df.loc[i])
  lst2 = {"GPS Time": lst["GPS Time"], "Device Time": lst["Device Time"]}
  lst = {i: float(lst[i]) if lst[i] != "-" else np.nan for i in list(lst.keys())[2:]}
  lstall = {**lst2, **lst}
  df.loc[i] = lstall
df



,GPS Time,Device Time,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),Engine RPM(rpm),Speed (OBD)(km/h),Acceleration Sensor(Total)(g),Litres Per 100 Kilometer(Instant)(l/100km),Speed (GPS)(km/h),Acceleration Sensor(Z axis)(g),Time(s)
0,Fri Mar 25 17:18:15 GMT+03:00 2016,25-Mar-2016 17:18:08.672,37.583105,55.848558,1.0,14.0,143.0,0.0,-1.069122,8.187592,4.949341,0.002015,NaN,0.0,0.002015,NaN,3.6,0.405627,62288.67
1,Fri Mar 25 17:18:16 GMT+03:00 2016,25-Mar-2016 17:18:08.767,37.583102,55.848554,1.0,14.0,143.0,0.0,-1.529938,8.353485,5.078201,0.027657,NaN,0.0,0.027657,NaN,3.6,0.417656,62288.77
2,NaN,25-Mar-2016 17:18:08.867,NaN,NaN,NaN,14.0,143.0,0.0,-1.483780,8.233429,4.653717,0.016112,NaN,0.0,0.016112,NaN,NaN,0.374385,62288.87
3,NaN,25-Mar-2016 17:18:08.966,NaN,NaN,NaN,14.0,143.0,0.0,-1.489288,8.442078,4.666107,0.015226,NaN,0.0,0.015226,NaN,NaN,0.375648,62288.97
4,NaN,25-Mar-2016 17:18:09.067,NaN,NaN,NaN,14.0,143.0,0.0,-0.720825,8.725891,4.743317,0.034077,NaN,0.0,0.034077,NaN,NaN,0.383519,62289.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22276,NaN,25-Mar-2016 17:55:21.259,NaN,NaN,NaN,4.0,166.0,161.2,0.998703,9.563507,1.149261,0.006151,599.50,0.0,0.006151,NaN,NaN,0.017152,64521.26
22277,NaN,25-Mar-2016 17:55:21.358,NaN,NaN,NaN,4.0,166.0,161.2,1.070480,9.641998,1.154083,0.006844,601.25,0.0,0.006844,NaN,NaN,0.017644,64521.36
22278,NaN,25-Mar-2016 17:55:21.458,NaN,NaN,NaN,4.0,166.0,161.2,1.026428,9.537384,1.169113,0.006450,601.25,0.0,0.006450,NaN,NaN,0.019176,64521.46
22279,NaN,25-Mar-2016 17:55:21.557,NaN,NaN,NaN,4.0,166.0,161.2,0.681564,8.894714,1.603546,-0.057069,609.00,0.0,-0.057069,NaN,NaN,0.063460,64521.56


In [60]:
import folium
def create_map(x, y):
  latitude = y
  longitude = x

  position = list(zip(latitude,longitude))
  # создаем карту
  mymap = folium.Map(location=(0,0), zoom_start=16)
  # Добавляем границы карты включающий две точки максимумы и минимумы широты и долготы
  bounds = [(min(latitude),min(longitude)),(max(latitude),max(longitude))]
  mymap.fit_bounds(bounds)
  # # Добавляем путь на карту, соединяющий все точки
  folium.PolyLine(locations = position, color='#E04100').add_to(mymap)
  return mymap


In [100]:
def start_kalman(df):
    v = decompose_vel(df.loc[0,'GPS Speed (Meters/second)'],df.loc[0,'Bearing'])
    x_0 = np.array([df.loc[0,"Longitude"],df.loc[0,"Latitude"],v[0],v[1]])
    sigma_lambda = 0.00001
    sigma_phi = 0.00001
    sigma_v_lambda = 0.1
    sigma_v_phi = 0.1
    P_0 = np.array(
        [[sigma_lambda**2,0,0,0],
         [0,sigma_lambda**2,0,0],
         [0,0,sigma_v_phi**2,0 ],
         [0,0,0,sigma_v_phi**2 ]]
    )
    return x_0, P_0

def matrix_F(df, k):
    a = np.zeros((4, 4))
    RPMk = df.loc[k, "Engine RPM(rpm)"]
    RPMk_1 = df.loc[k-1, "Engine RPM(rpm)"]
    if np.isnan(RPMk) or np.isnan(RPMk_1):
        RPMk = 1
        RPMk_1 = 1
    if RPMk_1 == 0:
        RPMk = 1
        RPMk_1 = 1

    O = np.deg2rad(float(df.loc[k, "Bearing"]) - float(df.loc[k-1, "Bearing"]))
    R3 = 6371221 #Радиус земли в м
    dt = float(df.loc[k, "Time(s)"]) - float(df.loc[k-1, "Time(s)"])
    if O != 0:
        a[0][0] = 1
        a[0][1] = 0
        a[0][2] = (RPMk * dt)/(R3*RPMk_1 * O**2) * (O*sin(O) + cos(O)-1)
        a[0][3] = -(RPMk * dt)/(R3*RPMk_1 * O**2) * (sin(O)- O*cos(O))

        a[1][0] = 0
        a[1][1] = 1
        a[1][2] = (RPMk * dt)/(R3*RPMk_1 * O**2) * (sin(O) - O*cos(O))
        a[1][3] = (RPMk * dt)/(R3*RPMk_1 * O**2) * (O*sin(O) + cos(O)-1)

        a[2][0] = 0
        a[2][1] = 0
        a[2][2] =  cos(O)
        a[2][3] = -RPMk/RPMk_1 * sin(O)

        a[3][0] = 0
        a[3][1] = 0
        a[3][2] = RPMk/RPMk_1 * sin(O)
        a[3][3] = RPMk/RPMk_1 * cos(O)
    else:
        a[0][0] = 1
        a[0][1] = 0
        a[0][2] = (RPMk * dt)/(R3*RPMk_1 * 2)
        a[0][3] = 0

        a[1][0] = 0
        a[1][1] = 1
        a[1][2] = 0
        a[1][3] = (RPMk * dt)/(R3*RPMk_1 * 2)

        a[2][0] = 0
        a[2][1] = 0
        a[2][2] = RPMk/RPMk_1
        a[2][3] = 0

        a[3][0] = 0
        a[3][1] = 0
        a[3][2] = 0
        a[3][3] = RPMk/RPMk_1
    return a

def matrix_Q(df,k):
    dt = df.loc[k,"Time(s)"] - df.loc[k-1,"Time(s)"]
    sigma_lambda_env = 0.0001*dt
    sigma_phi_env = 0.0001*dt
    sigma_v_lambda_env = 0.05*dt
    sigma_v_phi_env = 0.05*dt
    a = np.zeros((4, 4))
    a[0][0] = sigma_lambda_env**2
    a[1][1] = sigma_phi_env**2
    a[2][2] = sigma_v_lambda_env**2
    a[3][3] = sigma_v_phi_env**2
    return a

def matrix_R(df,k):
    angle = df.loc[k,'Bearing']
    sigma_meas_lambda = 0.00001
    sigma_meas_phi = 0.00001
    sigma_meas_v_GPS2 = 0.1**2
    sigma_meas_v_OBD2 = (0.1*1000/3600)**2
    K = sigma_meas_v_GPS2/(sigma_meas_v_GPS2+sigma_meas_v_OBD2)
    sigma_v2 = sigma_meas_v_GPS2 - K*sigma_meas_v_GPS2
    if np.isnan(df.loc[k,"GPS Speed (Meters/second)"]):
        sigma_v2 = sigma_meas_v_OBD2
    if np.isnan(df.loc[k,"Speed (OBD)(km/h)"]):
        sigma_v2 = sigma_meas_v_GPS2
    sigma_v2_lambda = decompose_vel(np.sqrt(sigma_v2),angle)[0]**2
    sigma_v2_phi = decompose_vel(np.sqrt(sigma_v2),angle)[1]**2
    a = np.zeros((4, 4))
    a[0][0] = sigma_meas_lambda**2
    a[1][1] = sigma_meas_phi**2
    a[2][2] = sigma_v2_lambda
    a[3][3] = sigma_v2_phi
    return a

def matrix_H(df,k):
    a = np.eye(4)
    return a

def matrix_z(df,k):
    angle = df.loc[k,'Bearing']
    lambda_k = df.loc[k,"Longitude"]
    phi_k = df.loc[k,"Latitude"]
    sigma_meas_v_GPS2 = 0.1**2
    sigma_meas_v_OBD2 = (0.1*1000/3600)**2 #м/c
    K = sigma_meas_v_GPS2/(sigma_meas_v_GPS2+sigma_meas_v_OBD2)
    v_k_GPS = df.loc[k,'GPS Speed (Meters/second)']
    v_k_OBD = df.loc[k,'Speed (OBD)(km/h)']*1000/3600 #м/c

    v_k = v_k_GPS + K*(v_k_OBD-v_k_GPS)
    if np.isnan(df.loc[k,"GPS Speed (Meters/second)"]):
        v_k = df.loc[k,"Speed (OBD)(km/h)"]
    if np.isnan(df.loc[k,"Speed (OBD)(km/h)"]):
        v_k = df.loc[k,"GPS Speed (Meters/second)"]
    v = decompose_vel(v_k,angle)
    v_lambda_k = v[0]
    v_phi_k = v[1]
    return np.array([[lambda_k],[phi_k],[v_lambda_k],[v_phi_k]])

class Kalman:
    def __init__(self,x_hat,P_0):
        self.x_hat = np.full((x_hat.shape[0],1),0.)
        for i in range(len(x_hat)):
            self.x_hat[i,0] = x_hat[i]
        self.P = P_0

    def predict(self,F,Q,B=np.nan,u=np.nan):
        if np.isnan(B) or np.isnan(u):
            B = np.full(F.shape, 0)
            u = np.full(self.x_hat.shape, 0)

        self.x_hat = F.dot(self.x_hat) + B.dot(u)
        self.P = F.dot(self.P).dot(F.T) + Q

    def get_estimate(self):
        return [self.x_hat, self.P]

    def update(self,H,R,z):
        K_ = self.P.dot(H.T).dot(np.linalg.inv(H.dot(self.P).dot(H.T)+R))
        self.x_hat = self.x_hat + K_.dot(z-H.dot(self.x_hat))
        self.P = self.P-K_.dot(H).dot(self.P)


In [103]:
#Главный цикл
kalman = Kalman(start_kalman(df)[0],start_kalman(df)[1])
data_predict_lambda = np.array([])
data_predict_phi = np.array([])
data_update_lambda = np.array([])
data_update_phi = np.array([])
for i in range(1,len(df)):
    F = matrix_F(df,i)
    Q = matrix_Q(df,i)
    kalman.predict(F=F,Q=Q)
    data_predict_lambda = np.append(data_predict_lambda,kalman.get_estimate()[0][0])
    data_predict_phi = np.append(data_predict_phi,kalman.get_estimate()[0][1])
    if np.isnan(df.loc[i,"Longitude"]) or (np.isnan(df.loc[i,"Speed (OBD)(km/h)"]) and np.isnan(df.loc[i,"GPS Speed (Meters/second)"])):
        pass
    else:
        z = matrix_z(df,i)
        H = matrix_H(df,i)
        R = matrix_R(df,i)
        kalman.update(H=H,R=R,z=z)
        data_update_lambda = np.append(data_update_lambda,kalman.get_estimate()[0][0])
        data_update_phi = np.append(data_update_phi,kalman.get_estimate()[0][1])

xhat =  [[ 3.75831054e+01]
 [ 5.58485583e+01]
 [ 6.12323400e-17]
 [-1.00000000e+00]]


In [102]:
create_map(data_predict_lambda, data_predict_phi)

In [104]:
create_map(data_update_lambda, data_update_phi)

Напоминаю, что нам надо реализовать 4-мерный фильтр Калмана со следующими уравнениями:
$$\color{#ff035f} \hat{x}_k \color{white}= F_k \color{#2771e8} \hat{x}_{k-1} + \color{white} B_k \color{#2ecbff} \color{#ff861c}\vec{u}_k$$
$$\color{#ff035f} P_k \color{white} = F_k \color{#2771e8} P_{k-1} \color{white}F_k^T + \color{#02a879}Q_k$$
$$\color{#9a02a8} K' \color{white} = \color{#ff035f}P_k \color{white} H_k^T \left( H_k \color{#ff035f}P_k \color{white} H_k^T  + \color{#02a879}R_k\color{white}\right)^{-1}$$
$$\color{#2771e8} \hat{x}'_k \color{white} = \color{#ff035f} \hat{x}_k \color{white} + \color{#9a02a8} K' \color{white} \left( \color{#9aa802} \vec{z}_k \color{white} - H_k \color{#ff035f} \hat{x}_k \color{white} \right)$$
$$\color{#2771e8} P'_k \color{white} = \color{#ff035f} P_k \color{white} - \color{#9a02a8} K' \color{white} H_k \color{#ff035f} P_k$$

Вектор состояния $\hat{x} _k = \left( \begin{matrix} \lambda_k \\ \phi_k \\ v_{\lambda k} \\ v_{\phi k} \\ \end{matrix} \right)$.
# Predict
## Разложение вектора скорости $\vec{v_k}$

$\lambda_k$ и $\phi_k$ измеряются в градусах, а $v_{\lambda k}$ и $v_{\phi k}$ – в $м/c$.
Посмотрим, как меняется $v_\lambda$: ($\lambda_k$ – долгота (вдоль параллели), $\phi_k$ – широта (вдоль меридиана))

![](./Source/Speed_RPM_Bearing.png)

Угол $\theta$ всегда ориентирует относительно севера (т.е. относительно меридиана, на котором в данный момент измеряется ориентация). Таким образом, $v_\lambda$ всегда отвечает за движение вдоль параллели (меняется долгота), а $v_\phi$ – за движение по меридиану (который со временем меняется) (меняется широта).

Предполагается, что скорость равномерно растет в $\frac{RPM_k}{RPM_{k-1}}$ раз по модулю и вектор скорости при этом равномерно поворачивается на угол $\theta = Beaing_k - Bearing_{k-1}$. Тогда $$\vec{v} _k = \left( \begin{matrix} v_{\lambda k} \\ v_{\phi k} \\ \end{matrix} \right) = \frac{RPM_k}{RPM_{k-1}}\left( \begin{matrix} \cos \theta & -\sin \theta \\ \sin \theta & \cos \theta \\ \end{matrix} \right) \left( \begin{matrix} v_{\lambda k-1} \\ v_{\phi k-1} \\ \end{matrix} \right).$$
Т.е. $$v_{\lambda k} = \frac{RPM_k}{RPM_{k-1}} \left( v_{\lambda k-1}\cos{\theta} - v_{\phi k-1}\sin{\theta} \right);$$
$$v_{\phi k} = \frac{RPM_k}{RPM_{k-1}} \left( v_{\lambda k-1}\sin{\theta} + v_{\phi k-1}\cos{\theta} \right).$$

## Учет влияния скорости на положение в модели системы

### Общие рассуждения

Движение по меридиану ($v_\phi$) происходит по окружности с постоянным радиусом $R_з$, которая поворачивается вокруг оси $SN$ (Юг-СЕВЕР) из-за движения вдоль параллелей.
![[GEO_Latitude.png]]

Таким образом, чтобы определить изменение широты, нужно проинтегрировать $d\phi$:
$$\Delta \phi = \int\limits_0^{\Delta t}d\phi$$$$R_з d\phi = v_\phi(t)dt \Rightarrow \Delta \phi = \frac{1}{R_з}\int\limits_0^{\Delta t} v_\phi (t) dt$$
Таким образом
$$\phi _k = \phi _{k-1} + \frac{1}{R_з}\int\limits_0^{\Delta t} v_{\phi k} (t) dt.$$
Движение по параллелям осуществляется несколько сложнее: Это по сути движение по окружности, которая непрерывно во времени меняет свой радиус.
![[GEO_Longitude.png]]
$$R(t)=R_з \cdot \cos \phi(t)$$
$$\phi_k(t) = \phi _{k-1} + \frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (\xi) d\xi$$
$$R_k(t)=R_з \cdot \cos \left( \phi _{k-1} + \frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (\xi) d\xi \right)$$
Тогда $\lambda _k$ находится путем интегрирования:

$$\lambda _k = \lambda _{k-1} + \int\limits_0^{\Delta t} \frac{v_{\lambda k}(t)}{R_k(t)}dt = \lambda _{k-1} + \frac{1}{R_з}\int\limits_0^{\Delta t} \frac{v_{\lambda k}(t)}{\cos \left( \phi _{k-1} + \frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (\xi) d\xi \right)}dt.$$

### Нахождение интегралов

Необходимо получить $v_{\lambda k}(t)$ и $v_{\phi k}(t)$ при $t \in [0;\Delta t]$:
$$v_{\phi k}(t) = \frac{RPM_k \cdot t}{RPM_{k-1} \cdot \Delta t} \left( v_{\lambda k-1}\sin{\frac{\theta \cdot t}{\Delta t}} + v_{\phi k-1}\cos{\frac{\theta \cdot t}{\Delta t}} \right);$$$$v_{\lambda k}(t) = \frac{RPM_k \cdot t}{RPM_{k-1} \cdot \Delta t} \left( v_{\lambda k-1}\cos{\frac{\theta \cdot t}{\Delta t}} - v_{\phi k-1}\sin{\frac{\theta \cdot t}{\Delta t}} \right).$$
Теперь найдем интегралы $\frac{1}{R_з}\int\limits_0^{\Delta t} v_{\phi k} (t) dt$ и $\int\limits_0^{\Delta t} \frac{v_{\lambda k}(t)}{R_k(t)}dt$:

#### Широта $\phi$

Вычислим $\frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (x) dx$.
$$\int\limits_0^{t} v_{\phi k}(x)dx = \frac{RPM_k}{RPM_{k-1} \cdot \Delta t}\int\limits_0^{t}x \left( v_{\lambda k-1}\sin{\frac{\theta \cdot t}{\Delta t}} + v_{\phi k-1}\cos{\frac{\theta \cdot t}{\Delta t}} \right)dx = \frac{RPM_k \cdot (\Delta t)^2}{RPM_{k-1} \cdot \Delta t \cdot \theta^2} \left[ v_{\lambda k-1}\int\limits_0^{t}\frac{\theta \cdot x}{\Delta t}\sin{\frac{\theta \cdot x}{\Delta t}}d \left(\frac{\theta \cdot x}{\Delta t}\right) + v_{\phi k-1}\int\limits_0^{t} \frac{\theta \cdot x}{\Delta t}\cos{\frac{\theta \cdot x}{\Delta t}}d\left(\frac{\theta \cdot x}{\Delta t}\right) \right]=$$
$$=\frac{RPM_k \cdot \Delta t}{RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1}\int\limits_0^{t}\frac{\theta \cdot x}{\Delta t}\sin{\frac{\theta \cdot x}{\Delta t}}d \left(\frac{\theta \cdot x}{\Delta t}\right) + v_{\phi k-1}\int\limits_0^{t} \frac{\theta \cdot x}{\Delta t}\cos{\frac{\theta \cdot x}{\Delta t}}d\left(\frac{\theta \cdot x}{\Delta t}\right) \right].$$
Произведем замену $n = \frac{x \cdot \theta}{\Delta t}$, тогда пределы интегрирования изменятся следующим образом: $\left[ \begin{matrix} 0 \rightarrow 0 \\ t \rightarrow \frac{t \cdot \theta}{\Delta t} \\ \end{matrix} \right]$ и мы получим
$$\int\limits_0^{t} v_{\phi k}(x)dx = \frac{RPM_k \cdot \Delta t}{RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1}\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\sin{n}\text{ }dn + v_{\phi k-1}\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\cos{n} \text{ }dn \right].$$
Интегралы $\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\sin{n}\text{ }dn$ и $\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\cos{n}\text{ }dn$ легко берутся по частям, и они равны
$$\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\sin{n}\text{ }dn = \sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right);$$$$\int\limits_0^{\frac{t \cdot \theta}{\Delta t}}n\cos{n}\text{ }dn = \frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1.$$
Подставляем:
$$\int\limits_0^{t} v_{\phi k}(x)dx = \frac{RPM_k \cdot \Delta t}{RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right) \right) + v_{\phi k-1} \left(\frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1\right)  \right].$$
Тогда
$$\frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (x) dx = \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right) \right) + v_{\phi k-1} \left(\frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1\right)  \right]$$
и
$$\phi _k = \phi _{k-1} + \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \theta - \theta \cos \theta \right) + v_{\phi k-1} \left(\theta \sin \theta + \cos \theta -1\right)  \right].$$

#### Долгота $\lambda$

Вычислим $\int\limits_0^{\Delta t} \frac{v_{\lambda k}(t)}{R_k(t)}dt$.
$$\int\limits_0^{\Delta t} \frac{v_{\lambda k}(t)}{R_k(t)}dt =\frac{RPM_k}{R_з \cdot RPM_{k-1} \cdot \Delta t}\int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{\frac{\theta \cdot t}{\Delta t}} - v_{\phi k-1}\sin{\frac{\theta \cdot t}{\Delta t}} \right)}{\cos \left( \phi _{k-1} + \frac{1}{R_з}\int\limits_0^{t} v_{\phi k} (\xi) d\xi \right)}dt = $$
$$=\frac{RPM_k}{R_з \cdot RPM_{k-1} \cdot \Delta t}\int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{\frac{\theta \cdot t}{\Delta t}} - v_{\phi k-1}\sin{\frac{\theta \cdot t}{\Delta t}} \right)}
{\cos \left( \phi _{k-1} + \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right) \right) + v_{\phi k-1} \left(\frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1\right)  \right]\right)}dt
$$
Отсюда получаем
$$\lambda _k = \lambda _{k-1} + \frac{RPM_k}{R_з \cdot RPM_{k-1} \cdot \Delta t}\int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{\frac{\theta \cdot t}{\Delta t}} - v_{\phi k-1}\sin{\frac{\theta \cdot t}{\Delta t}} \right)}{\cos \left( \phi _{k-1} + \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right) \right) + v_{\phi k-1} \left(\frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1\right)  \right]\right)}dt.$$
Несложно заметить, что получить представление в виде линейной комбинации $v_{\lambda k-1}$, $v_{\phi k-1}$ и $\phi_{k-1}$ НЕВОЗМОЖНО (по крайней мере ОЧЕНЬ сложно), так что приходится прибегать к линеаризации (Разложение этого интеграла в многомерный ряд Тейлора). Для начала сделаем пару замен:$$\left[\begin{matrix} k_1 = \frac{\theta}{\Delta t} \\k_2 = \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \\ \end{matrix} \right]$$
Тогда получим
$$\mathcal{J} = \int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{\frac{\theta \cdot t}{\Delta t}} - v_{\phi k-1}\sin{\frac{\theta \cdot t}{\Delta t}} \right)}{\cos \left( \phi _{k-1} + \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left[ v_{\lambda k-1} \left(\sin \left(\frac{t \cdot \theta}{\Delta t} \right) - \frac{t \cdot \theta}{\Delta t} \cos \left(\frac{t \cdot \theta}{\Delta t} \right) \right) + v_{\phi k-1} \left(\frac{t \cdot \theta}{\Delta t} \sin \left(\frac{t \cdot \theta}{\Delta t} \right) + \cos \left(\frac{t \cdot \theta}{\Delta t} \right) -1\right)  \right]\right)}dt =$$
$$= \int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{k_1t} - v_{\phi k-1}\sin{k_1t} \right)}{\cos \left( \phi _{k-1} + k_2 \left[ v_{\lambda k-1} \left(\sin {k_1t} - {k_1t} \cos {k_1t} \right) + v_{\phi k-1} \left({k_1t} \sin {k_1t} + \cos {k_1t} -1\right)  \right]\right)}dt =$$
$$= \int\limits_0^{\Delta t} \frac{ t\left( v_{\lambda k-1}\cos{k_1t} - v_{\phi k-1}\sin{k_1t} \right)}{\cos \left[ \phi _{k-1} + v_{\lambda k-1} k_2\left(\sin {k_1t} - {k_1t} \cos {k_1t} \right) + v_{\phi k-1} k_2\left({k_1t} \sin {k_1t} + \cos {k_1t} -1\right) \right]}dt.$$
Введем $$f(\phi_{k-1},v_{\lambda k-1}, v_{\phi k-1}) = \frac{ t\left( v_{\lambda k-1}\cos{k_1t} - v_{\phi k-1}\sin{k_1t} \right)}{\cos \left[ \phi _{k-1} + v_{\lambda k-1} k_2\left(\sin {k_1t} - {k_1t} \cos {k_1t} \right) + v_{\phi k-1} k_2\left({k_1t} \sin {k_1t} + \cos {k_1t} -1\right) \right]}.$$
Мы стараемся сделать несколько вещей:
1. Добиться максимальной точности линеаризации, т.е. взять начальную точку, наиболее близкую к тем, что мы имеем в данных;
2. После линеаризации не иметь свободных (относительно $\phi_{k-1},v_{\lambda k-1}, v_{\phi k-1}$) членов, т.к. они никак не будут учтены в матрице ковариации и их придется пихать в управляющий сигнал, что не корректно.
В приоритете 2-й пункт:
Разлагая $f$ до 1го порядка вокруг $\left(\phi_{k-1}^*,v_{\lambda k-1}^*,v_{\phi k-1}^*\right)$, получим
$$f(\phi_{k-1},v_{\lambda k-1}, v_{\phi k-1}) \approx f\left(\phi_{k-1}^*,v_{\lambda k-1}^*,v_{\phi k-1}^*\right) + \frac{\partial f\left(\phi_{k-1}^*,v_{\lambda k-1}^*,v_{\phi k-1}^*\right)}{\partial \phi_{k-1}}(\phi_{k-1}-\phi_{k-1}^*) +  \frac{\partial f\left(\phi_{k-1}^*,v_{\lambda k-1}^*,v_{\phi k-1}^*\right)}{\partial v_{\lambda k-1}}(v_{\lambda k-1}-v_{\lambda k-1}^*) + \frac{\partial f\left(\phi_{k-1}^*,v_{\lambda k-1}^*,v_{\phi k-1}^*\right)}{\partial v_{\phi k-1}}(v_{\phi k-1}-v_{\phi k-1}^*).$$
Проще всего занулить все свободные слагаемые, положив $\left[ \begin{matrix} \phi_{k-1}^* = 0 \\ v_{\lambda k-1}^* = 0 \\ v_{\phi k-1}^* = 0 \\ \end{matrix}\right]$, т.к. тогда не будет свободно болтающихся частных производных и $f(0,0,0) = 0$, как можно видеть выше.
Осталось найти частные производные (промежуточные этапы опущены, при необходимости дергать @SaprZheks):
$$\begin{matrix}
\left. \frac{\partial f}{\partial \phi_{k-1}} \right|_{(0,0,0)} =  0\\
\left. \frac{\partial f}{\partial v_{\lambda k-1}}\right|_{(0,0,0)} = t \cos (k_1t)\\
\left. \frac{\partial f}{\partial v_{\phi k-1}} \right|_{(0,0,0)} = -t \sin (k_1t)\\
\end{matrix}$$
Разложим $f$ в многомерный ряд Тейлора в окрестности $\left(0,0,0 \right)$:
$$f(\phi_{k-1},v_{\lambda k-1}, v_{\phi k-1}) \approx t \cos (k_1t)v_{\lambda k-1} - t \sin (k_1t)v_{\phi k-1}.$$
Тогда можно без труда посчитать интеграл $\mathcal{J}$:
$$\mathcal{J} = \int\limits_0^{\Delta t} f(\phi_{k-1},v_{\lambda k-1}, v_{\phi k-1})dt \approx \int\limits_0^{\Delta t} \left[ t \cos (k_1 t) v_{\lambda k-1} - t \sin (k_1t)v_{\phi k-1} \right]dt = \frac{v_{\lambda k-1}}{k_1^2}\int\limits_0^{\Delta t}k_1t \cos (k_1 t) d(k_1t) - \frac{v_{\phi k-1}}{k_1^2}\int\limits_0^{\Delta t} k_1 t \sin (k_1 t) d(k_1 t).$$
Производя замену $n = k_1 t$, получим $\left[ \begin{matrix} 0 \rightarrow 0 \\ \Delta t \rightarrow k_1 \Delta t \\ \end{matrix} \right]$ и
$$\mathcal{J} \approx \frac{v_{\lambda k-1}}{k_1^2}\int\limits_0^{k_1 \Delta t}n \cos n \text{ }d(n) - \frac{v_{\phi k-1}}{k_1^2}\int\limits_0^{k_1 \Delta t} n \sin n \text{ }d(n).$$
Эти интегралы уже вычислялись ранее. Подставляя те результаты, получим
$$\mathcal{J} \approx \frac{v_{\lambda k-1}}{k_1^2} \left( k_1 \Delta t \sin (k_1 \Delta t) + \cos (k_1 \Delta t) -1 \right) - \frac{v_{\phi k-1}}{k_1^2} \left( \sin(k_1 \Delta t) - k_1 \Delta t \cos (k_1 \Delta t) \right).$$
$$\lambda _k \approx \lambda _{k-1} + \frac{RPM_k}{R_з \cdot RPM_{k-1} \cdot \Delta t} \left[\frac{v_{\lambda k-1}}{k_1^2} \left( k_1 \Delta t \sin (k_1 \Delta t) + \cos (k_1 \Delta t) -1 \right) - \frac{v_{\phi k-1}}{k_1^2} \left( \sin(k_1 \Delta t) - k_1 \Delta t \cos (k_1 \Delta t) \right) \right],$$
где $k_1 = \frac{\theta}{\Delta t}$. Подставляем:
$$\lambda _k \approx \lambda _{k-1} + \frac{RPM_k}{R_з \cdot RPM_{k-1} \cdot \Delta t} \left[\frac{v_{\lambda k-1}}{\left( \frac{\theta}{\Delta t}\right)^2} \left( \frac{\theta}{\Delta t} \Delta t \sin (\frac{\theta}{\Delta t} \Delta t) + \cos (\frac{\theta}{\Delta t} \Delta t) -1 \right) - \frac{v_{\phi k-1}}{\left(\frac{\theta}{\Delta t}\right)^2} \left( \sin(\frac{\theta}{\Delta t} \Delta t) - \frac{\theta}{\Delta t} \Delta t \cos (\frac{\theta}{\Delta t} \Delta t) \right) \right] = $$
$$=\lambda _{k-1} + \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1} \cdot \theta^2}\left[v_{\lambda k-1}\left( \theta \sin \theta + \cos \theta -1 \right) - v_{\phi k-1} \left( \sin\theta - \theta \cos \theta \right) \right].$$
Этот результат ровно такой же, как если бы мы положили, что параллели ведут себя как меридианы, т.е. вычисляли бы долготу так же, как и широту.

## Итоговая матрица эволюция системы

$$\lambda _k =\lambda _{k-1} + v_{\lambda k-1}\frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1} \cdot \theta^2}\left( \theta \sin \theta + \cos \theta -1 \right) - v_{\phi k-1} \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1} \cdot \theta^2}\left( \sin\theta - \theta \cos \theta \right) .$$
$$\phi _k = \phi _{k-1} +  v_{\lambda k-1}\frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2} \left(\sin \theta - \theta \cos \theta \right) + v_{\phi k-1} \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}\left(\theta \sin \theta + \cos \theta -1\right) .$$
$$v_{\lambda k} = v_{\lambda k-1}\frac{RPM_k}{RPM_{k-1}}\cos{\theta} - v_{\phi k-1}\frac{RPM_k}{RPM_{k-1}}\sin{\theta};$$
$$v_{\phi k} = v_{\lambda k-1}\frac{RPM_k}{RPM_{k-1}}\sin{\theta} + v_{\phi k-1}\frac{RPM_k}{RPM_{k-1}}\cos{\theta}.$$
Т.е. в матричной форме это выглядит так:
$$\hat{x} _k = \left( \begin{matrix} \lambda _k \\ \phi _k \\ v_{\lambda k} \\ v_{\phi k} \\ \end{matrix} \right) = \left( \begin{matrix}1 & 0 & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\theta \sin \theta + \cos \theta -1) & - \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\sin \theta - \theta \cos \theta) \\0 & 1 & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\sin \theta - \theta \cos \theta) & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\theta \sin \theta + \cos \theta -1) \\ 0 & 0 & \frac{RPM_k}{RPM_{k-1}}\cos{\theta} & -\frac{RPM_k}{RPM_{k-1}}\sin{\theta} \\ 0 & 0 & \frac{RPM_k}{RPM_{k-1}}\sin{\theta} & \frac{RPM_k}{RPM_{k-1}}\cos{\theta} \\\end{matrix} \right) \left( \begin{matrix} \lambda _{k-1} \\\phi _{k-1} \\ v_{\lambda k-1} \\ v_{\phi k-1} \\ \end{matrix} \right),$$
т.е.
$$F_k = \left( \begin{matrix}1 & 0 & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\theta \sin \theta + \cos \theta -1) & - \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\sin \theta - \theta \cos \theta) \\0 & 1 & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\sin \theta - \theta \cos \theta) & \frac{RPM_k \cdot \Delta t}{R_з \cdot RPM_{k-1}\cdot \theta^2}(\theta \sin \theta + \cos \theta -1) \\ 0 & 0 & \frac{RPM_k}{RPM_{k-1}}\cos{\theta} & -\frac{RPM_k}{RPM_{k-1}}\sin{\theta} \\ 0 & 0 & \frac{RPM_k}{RPM_{k-1}}\sin{\theta} & \frac{RPM_k}{RPM_{k-1}}\cos{\theta} \\\end{matrix} \right).$$
Что касается $B_k$ – у нас нет 100% данных управления $\Rightarrow$ $B_k = 0$

Матрица $P_0$ принимается диагональной (на диагонали дисперсии величин, измеренные датчиками), ибо в самый начальный момент времени скорость никак не влияет на уверенность в нашем положении (ибо наше начальное положение – чисто измерения, а не оценка), зато после нескольких итераций матрица $F_k$ скорректирует это и положение станет явно зависеть от скорости.

Аналогично дополнительную неопределенность системы описываем диагональной ковариационной матрицей $\color{#02a879}Q_k$, потому что это по сути допуски на параметры вектора состояния модели, и точно предсказать, как именно, например, рандомное изменение скорости может рандомно изменить положение, сказать сложно, проще заложить такие возможности в допуск положения. Таким образом, нужно просто оценить 4 среднеквадратических отклонения:
$$\color{#02a879}Q_k \color{white}= \left( \begin{matrix}
\sigma^2_\lambda  & 0 & 0 & 0 \\
0 & \sigma^2_\phi  & 0 & 0 \\
0 & 0 & \sigma^2_{v_\lambda} & 0 \\
0 & 0 & 0 & \sigma^2_{v_\phi } \\
\end{matrix}
\right).$$
Причем уместно предположить, что $\sigma_\lambda  = \sigma_\phi$ и $\sigma_{v_\lambda } = \sigma_{v_\phi }$.

# Update

За измерения возьмем столбец $\vec{z} _k = \left( \begin{matrix} \lambda \\ \phi \\ v_\lambda \\ v_\phi \\ \end{matrix} \right)$, $\lambda$ и $\phi$ в $\degree$ , $v_\lambda$ и $v_\phi$ в м/с с ковариацией $$R_k = \left( \begin{matrix} \sigma^2_{meas \lambda} & 0 & 0 & 0 \\
0 &  \sigma^2_{meas \phi} & 0 & 0 \\
0 & 0 & \sigma^2_{v_{meas \lambda}} & 0 \\
0 & 0 & 0 & \sigma^2_{v_{meas \phi}} \\
\end{matrix} \right).$$
Тогда ожидаемые измерения будут такими:
$$\vec{\mu} _{expected} = H_k \cdot \hat{x} _k,$$
где
$$H_k = \left( \begin{matrix} 1 & 0 & 0 & 0 \\ 0 &  1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ \end{matrix} \right)$$
и матрица ковариации
$$\Sigma_{expected} = H_k P_k H_k^T$$
$v_\lambda$ и $v_\phi$ берутся из 2-х показаний модуля скорости и ориентации $Bearing$.
В итоге получаем
$$\color{#9a02a8} K' \color{white} = \color{#ff035f}P_k \color{white} H_k^T \left( H_k \color{#ff035f}P_k \color{white} H_k^T  + \color{#02a879}R_k \color{white} \right)^{-1}$$
$$\color{#2771e8} \hat{x}'_k \color{white} = \color{#ff035f} \hat{x}_k \color{white} + \color{#9a02a8} K' \color{white} \left( \color{#9aa802} \vec{z}_k \color{white} - H_k \color{#ff035f} \hat{x}_k \color{white} \right)$$
$$\color{#2771e8} P'_k \color{white} = \color{#ff035f} P_k \color{white} - \color{#9a02a8} K' \color{white} H_k \color{#ff035f} P_k$$
